In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2024-03-13 14:30:23.657020: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 14:30:23.657095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 14:30:23.659758: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 14:30:23.678684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 14:30:26.102352: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels

# Get list of image paths
image_paths = list(paths.list_images("../Covid_X-Ray_Dataset/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
(800, 224, 224, 3)
(800,)


In [8]:
import gc
import os

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50" :
        labels = to_categorical(labels)

    else:
        data = data.reshape(data.shape[0],data.shape[1] * data.shape[2] * data.shape[3] )
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]


In [9]:
from sklearn.linear_model import LogisticRegression
# LR
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)



[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7375
Sensitivity: 0.775
Specificity: 0.7
AUC-ROC: 0.7374999999999999
MCC: 0.47634160003570514
Precision: 0.7209302325581395
F1 Score: 0.7469879518072289

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8625
Sensitivity: 0.8
Specificity: 0.925
AUC-ROC: 0.8625
MCC: 0.730731314484506
Precision: 0.9142857142857143
F1 Score: 0.8533333333333333

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.7375
Sensitivity: 0.625
Specificity: 0.85
AUC-ROC: 0.7374999999999999
MCC: 0.4875001055012461
Precision: 0.8064516129032258
F1 Score: 0.7042253521126761

[INFO] Fold 6 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7625
Sensitivity: 0.725
Specificity: 0.8
AUC-ROC: 0.7625000000000001
MCC: 0.5264828210920952
Precision: 0.7837837837837838
F1 Score: 0.7532467532467533

[INFO] Fold 7 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8375
Sensitivity: 0.775
Specificity: 0.9
AUC-ROC: 0.8374999999999999
MCC: 0.680336051416609
Precision: 0.8857142857142857
F1 Score: 0.8266666666666667

[INFO] Fold 8 / 10 for LR
Accuracy: 0.7875
Sensitivity: 0.75
Specificity: 0.825
AUC-ROC: 0.7874999999999999
MCC: 0.5766240421484852
Precision: 0.8108108108108109
F1 Score: 0.7792207792207791

[INFO] Fold 9 / 10 for LR
Accuracy: 0.8875
Sensitivity: 0.825
Specificity: 0.95
AUC-ROC: 0.8875
MCC: 0.7811265775524029
Precision: 0.9428571428571428
F1 Score: 0.88

[INFO] Fold 10 / 10 for LR
Accuracy: 0.7625
Sensitivity: 0.75
Specificity: 0.775
AUC-ROC: 0.7625
MCC: 0.5251641394443731
Precision: 0.7692307692307693
F1 Score: 0.7594936708860761


In [10]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.6875
Sensitivity: 0.75
Specificity: 0.625
AUC-ROC: 0.6875
MCC: 0.3779644730092272
Precision: 0.6666666666666666
F1 Score: 0.7058823529411765

[INFO] Fold 2 / 10 for NB
Accuracy: 0.675
Sensitivity: 0.725
Specificity: 0.625
AUC-ROC: 0.675
MCC: 0.35176323534072423
Precision: 0.6590909090909091
F1 Score: 0.6904761904761905

[INFO] Fold 3 / 10 for NB
Accuracy: 0.75
Sensitivity: 0.775
Specificity: 0.725
AUC-ROC: 0.75
MCC: 0.5006261743217589
Precision: 0.7380952380952381
F1 Score: 0.7560975609756099

[INFO] Fold 4 / 10 for NB
Accuracy: 0.675
Sensitivity: 0.725
Specificity: 0.625
AUC-ROC: 0.675
MCC: 0.35176323534072423
Precision: 0.6590909090909091
F1 Score: 0.6904761904761905

[INFO] Fold 5 / 10 for NB
Accuracy: 0.65
Sensitivity: 0.525
Specificity: 0.775
AUC-ROC: 0.65
MCC: 0.30983866769659335
Precision: 0.7
F1 Score: 0.6

[INFO] Fold 6 / 10 for NB
Accuracy: 0.7375
Sensitivity: 0.65
Specificity: 0.825
AUC-ROC: 0.7374999999999999
MCC: 0.48244488031307425
P

In [11]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.8375
Sensitivity: 0.775
Specificity: 0.9
AUC-ROC: 0.8374999999999999
MCC: 0.680336051416609
Precision: 0.8857142857142857
F1 Score: 0.8266666666666667

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7875
Sensitivity: 0.625
Specificity: 0.95
AUC-ROC: 0.7875
MCC: 0.6080062157172438
Precision: 0.9259259259259259
F1 Score: 0.7462686567164178

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.825
Sensitivity: 0.775
Specificity: 0.875
AUC-ROC: 0.825
MCC: 0.6532745799184878
Precision: 0.8611111111111112
F1 Score: 0.8157894736842106

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.8625
Sensitivity: 0.825
Specificity: 0.9
AUC-ROC: 0.8625
MCC: 0.7270477053176553
Precision: 0.8918918918918919
F1 Score: 0.8571428571428571

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.8
Sensitivity: 0.65
Specificity: 0.95
AUC-ROC: 0.7999999999999999
MCC: 0.628970902033151
Precision: 0.9285714285714286
F1 Score: 0.7647058823529412

[INFO] Fold 6 / 10 for KNN
Accuracy: 0.8625
Sensitivity: 0.775
Specificity: 0.9

In [12]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.825
Sensitivity: 0.775
Specificity: 0.875
AUC-ROC: 0.825
MCC: 0.6532745799184878
Precision: 0.8611111111111112
F1 Score: 0.8157894736842106

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.825
Sensitivity: 0.825
Specificity: 0.825
AUC-ROC: 0.825
MCC: 0.65
Precision: 0.825
F1 Score: 0.825

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.8625
Sensitivity: 0.875
Specificity: 0.85
AUC-ROC: 0.8625
MCC: 0.7252266687565153
Precision: 0.8536585365853658
F1 Score: 0.8641975308641976

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9
Sensitivity: 0.825
Specificity: 0.975
AUC-ROC: 0.8999999999999999
MCC: 0.8091547798786778
Precision: 0.9705882352941176
F1 Score: 0.8918918918918919

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.85
Sensitivity: 0.75
Specificity: 0.95
AUC-ROC: 0.85
MCC: 0.7144345083117603
Precision: 0.9375
F1 Score: 0.8333333333333334

[INFO] Fold 6 / 10 for Random Forest
Accuracy: 0.8125
Sensitivity: 0.75
Specificity: 0.875
AU

In [13]:


lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 360, number of negative: 360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 22.081474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19479745
[LightGBM] [Info] Number of data points in the train set: 720, number of used features: 150528
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

In [14]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the VGG16 model pre-trained on ImageNet data
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Build a new model using VGG16 as a feature extractor
with tf.device('/GPU:0'):
    model_vgg16 = Sequential()
    model_vgg16.add(vgg16_model)
    model_vgg16.add(Flatten())
    model_vgg16.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_vgg16.add(Dropout(0.5, seed = 0))
    model_vgg16.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_vgg16, 'VGG16', data, labels)

2024-03-13 15:56:19.809251: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37960 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0



[INFO] Fold 1 / 10 for VGG16


2024-03-13 15:56:21.495092: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20


2024-03-13 15:56:26.655242: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-13 15:56:26.967011: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 15:56:30.146047: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f416424da50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 15:56:30.146103: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-03-13 15:56:30.161307: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710345390.469499  529376 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 [==============================] - 8s 78ms/step - loss: 18.3075 - accuracy: 0.6931
Epoch 2/20
12/12 [==============================] - 1s 49ms/step - loss: 3.1726 - accuracy: 0.8903
Epoch 3/20
12/12 [==============================] - 1s 49ms/step - loss: 1.2481 - accuracy: 0.9403
Epoch 4/20
12/12 [==============================] - 1s 49ms/step - loss: 0.1816 - accuracy: 0.9681
Epoch 5/20
12/12 [==============================] - 1s 49ms/step - loss: 0.1226 - accuracy: 0.9778
Epoch 6/20
12/12 [==============================] - 1s 50ms/step - loss: 0.0751 - accuracy: 0.9847
Epoch 7/20
12/12 [==============================] - 1s 49ms/step - loss: 0.0582 - accuracy: 0.9931
Epoch 8/20
12/12 [==============================] - 1s 51ms/step - loss: 0.0360 - accuracy: 0.9875
Epoch 9/20
12/12 [==============================] - 1s 43ms/step - loss: 0.0542 - accuracy: 0.9847
Epoch 10/20
12/12 [==============================] - 1s 49ms/step - loss: 0.0343 - accuracy: 0.9889
Epoch 11/20
12/12 [

In [15]:

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the EfficientNetB7 model pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Build a new model using EfficientNetB7 as a feature extractor
with tf.device('/GPU:0'):
    model_efficientnet = Sequential()
    model_efficientnet.add(efficientnet_model)
    model_efficientnet.add(Flatten())
    model_efficientnet.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_efficientnet.add(Dropout(0.5, seed=0))
    model_efficientnet.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_efficientnet, 'EfficientNetB0', data, labels)



[INFO] Fold 1 / 10 for EfficientNetB0
Epoch 1/20
12/12 [==============================] - 18s 89ms/step - loss: 8.4246 - accuracy: 0.6764
Epoch 2/20
12/12 [==============================] - 1s 54ms/step - loss: 1.5859 - accuracy: 0.8667
Epoch 3/20
12/12 [==============================] - 1s 54ms/step - loss: 0.5283 - accuracy: 0.9361
Epoch 4/20
12/12 [==============================] - 1s 54ms/step - loss: 0.1536 - accuracy: 0.9625
Epoch 5/20
12/12 [==============================] - 1s 56ms/step - loss: 0.1031 - accuracy: 0.9639
Epoch 6/20
12/12 [==============================] - 1s 54ms/step - loss: 0.0501 - accuracy: 0.9833
Epoch 7/20
12/12 [==============================] - 0s 37ms/step - loss: 0.0523 - accuracy: 0.9778
Epoch 8/20
12/12 [==============================] - 1s 54ms/step - loss: 0.0387 - accuracy: 0.9875
Epoch 9/20
12/12 [==============================] - 0s 38ms/step - loss: 0.0427 - accuracy: 0.9847
Epoch 10/20
12/12 [==============================] - 1s 54ms/step - l

In [16]:
from tensorflow.keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the ResNet50 model pre-trained on ImageNet data
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build a new model using ResNet50 as a feature extractor
with tf.device('/GPU:0'):
    model_resnet50 = Sequential()
    model_resnet50.add(resnet50_model)
    model_resnet50.add(Flatten())
    model_resnet50.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_resnet50.add(Dropout(0.5, seed=0))
    model_resnet50.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_resnet50, 'ResNet50', data, labels)



[INFO] Fold 1 / 10 for ResNet50
Epoch 1/20
12/12 [==============================] - 11s 121ms/step - loss: 15.4705 - accuracy: 0.7153
Epoch 2/20
12/12 [==============================] - 1s 69ms/step - loss: 2.6501 - accuracy: 0.8972
Epoch 3/20
12/12 [==============================] - 1s 69ms/step - loss: 0.7698 - accuracy: 0.9417
Epoch 4/20
12/12 [==============================] - 1s 69ms/step - loss: 0.1985 - accuracy: 0.9708
Epoch 5/20
12/12 [==============================] - 1s 69ms/step - loss: 0.0642 - accuracy: 0.9819
Epoch 6/20
12/12 [==============================] - 1s 74ms/step - loss: 0.0134 - accuracy: 0.9944
Epoch 7/20
12/12 [==============================] - 1s 45ms/step - loss: 0.0313 - accuracy: 0.9861
Epoch 8/20
12/12 [==============================] - 1s 44ms/step - loss: 0.0157 - accuracy: 0.9958
Epoch 9/20
3/3 [==============================] - 2s 27ms/step
Accuracy: 0.9375
Sensitivity: 0.9
Specificity: 0.975
AUC-ROC: 0.9374999999999999
MCC: 0.8774713684868253
Prec

In [17]:
# Save results to a CSV file
results_df.to_csv('DS2_baseline.csv', index=False)

In [18]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.80750,0.7750,0.8400,0.80750,0.618506,0.831978,0.800805,137.824219,481.944550
1,NB,0.71875,0.7025,0.7350,0.71875,0.440826,0.729507,0.712702,115.093750,39.783407
2,KNN,0.82750,0.7425,0.9125,0.82750,0.666808,0.897236,0.810517,103.871094,16.731331
3,Random Forest,0.86500,0.8300,0.9000,0.86500,0.734277,0.894775,0.859484,151.519531,39.535012
4,LGBM,0.89250,0.8725,0.9125,0.89250,0.787257,0.909808,0.889737,3099.839844,4536.085926
5,VGG16,0.99375,0.9950,0.9925,0.99375,0.987527,0.992683,0.993827,776.316406,78.352104
6,EfficientNetB0,0.99000,0.9900,0.9900,0.99000,0.980000,0.990000,0.990000,199.777344,110.295233
7,ResNet50,0.99375,0.9900,0.9975,0.99375,0.987747,0.997297,0.993506,164.195312,98.037416
